Sorry, this is really rough, currently this has some initial EDA I did on the various subreddits I'm thinking of scraping as well as the backbone of a potential scraping function.

Some things I need to research and/or ask about

1) I'm currently thinking about saving this as a list of nested dictionaries. Need to determine if that's the best idea or not

2) I'm worried I'm hitting reddit too much (1,000 comments per 5 subs) * (up to 1,000 posts per unique user). Researched that a bit and I think praw has a built in 2 sec delay so things are probably okay but probably should research a bit more and break up my scrapes

3) Currently, my code gives equal weight to a comment poster but that's probably not ideal, especially since people can go onto a subreddit and comment even though they're not a supporter. Would ideally like to do something where I account for this.

4) My code currently doesn't have a way to append or add more data, need to consider how I might go about doing that


In [1]:
import praw
import json
import time
import pandas as pd
import sys
import numpy as np

In [2]:
sys.version

'3.7.3 (default, Mar 27 2019, 16:54:48) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
praw.__version__

'6.3.1'

In [4]:
json_file = open('Reddit_creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [5]:
reddit = praw.Reddit(user_agent=json_data['user_agent'],
                     client_id=json_data['client_id'], client_secret=json_data['client_secret'])

In [6]:
yang_sub = reddit.subreddit('YangForPresidentHQ')

In [7]:
pete_sub = reddit.subreddit('Pete_Buttigieg')

In [8]:
sanders_sub = reddit.subreddit('SandersForPresident')

In [9]:
harris_sub = reddit.subreddit('Kamala')

In [10]:
biden_sub = reddit.subreddit('JoeBiden')

In [11]:
subs = [yang_sub,pete_sub,sanders_sub,harris_sub,biden_sub]

In [12]:
users = []

In [13]:
start = time.time()
for sub in subs:
    for comment in sub.comments(limit=1000):
        users.append(comment.author)
        
print(len(users))
users = list(set(users))
print(len(users))
end = time.time()
print(end-start)

4668
1802
36.493520975112915


In [37]:
# nums = np.random.randint(1,1733,100)

In [39]:
# short_list = []
# for x in nums:
#     short_list.append(users[x])

In [1]:
# short_list

In [ ]:
# #code obtained from https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# # load additional module
# import pickle

# # define a list of places
# with open('users.data', 'wb') as filehandle:
#     # store the data as binary data stream
#     pickle.dump(set(users), filehandle)

In [ ]:
#code obtained from https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# with open('users.data', 'rb') as filehandle:
#     # read the data as binary data stream
#     test_set = pickle.load(filehandle)

In [ ]:
# small_list2 = list(set(users))[100:200]

In [ ]:
# len(small_list)

In [ ]:
#First full scrape from the evening of 8/6/19 - morning of 8/7/19
#Second full scrape starts @ 8/13/19 @ 10:41 AM - 1:21 PM

In [14]:
start = time.time()
text = []
author = []
subreddit = []
postid = []
counter = 0
for user in users:
    if counter % 10 == 0:
        prog = time.time()
        print(counter)
        print((prog-start)/60)
    try:  #handle cases where redditor is deleted or something
        for comment in user.comments.new(limit=1000):
            text.append(comment.body)
            author.append(comment.author)
            postid.append(comment.id)
            subreddit.append(comment.subreddit.display_name)
    except:
        pass
    counter += 1
end = time.time()
print(end-start)

0
3.3855438232421873e-06
10
0.7920368194580079
20
1.5936389327049256
30
2.6889748016993207
40
3.4561569849650065
50
4.342033966382345
60
5.1525907198588055
70
6.101528819402059
80
6.8767002860705055
90
7.4497390508651735
100
8.27935370206833
110
9.09846228758494
120
10.169164403279622
130
10.867194867134094
140
11.852298367023469
150
12.644133989016215
160
13.42756112019221
170
13.93772920370102
180
14.948122255007426
190
15.514604552586873
200
16.397117618719736
210
17.236179900169372
220
18.349484737714132
230
19.344518303871155
240
20.146442651748657
250
21.053411400318147
260
21.820803304513294
270
22.741350452105205
280
23.54286926984787
290
24.496900220712025
300
25.4795796195666
310
26.240101917584738
320
27.249991317590077
330
28.437754571437836
340
29.374432869752248
350
30.59151211977005
360
31.304382836818696
370
31.99634618361791
380
32.91611350377401
390
33.63564336697261
400
34.53578758637111
410
35.57668828566869
420
36.53078918457031
430
37.285315402348836
440
38.294005

In [15]:
df = pd.DataFrame([text,author,subreddit,postid],\
                        index=['Comment Text','Author','Subreddit','Post ID']).T

In [16]:
df.shape

(1156518, 4)

In [17]:
df.to_csv('full_scrape_2.csv',index=False)

In [ ]:
df.iloc[1,0]

In [ ]:

for user in set(users):
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if dict_sub.get(comment.subreddit.display_name,0) == 0:
            dict_sub[comment.subreddit.display_name] = 1
        else:
            dict_sub[comment.subreddit.display_name] += 1
    yang_dict[user] = dict_sub
#     print(f'finished {user}')
    time.sleep(5)

In [ ]:

yang_users = []
for comment in yang_sub.comments(limit=1000):
    yang_users.append(comment.author)
print(len(yang_users))
print(len(set(yang_users)))


In [ ]:
setyang = set(yang_users)

In [ ]:
harris_users = []
for comment in harris_sub.comments(limit=1000):
    harris_users.append(comment.author)
print(len(harris_users))
print(len(set(harris_users)))

In [ ]:
setharris = set(harris_users)

In [ ]:
biden_users = []
for comment in biden_sub.comments(limit=1000):
    biden_users.append(comment.author)
print(len(biden_users))
print(len(set(biden_users)))

In [ ]:
setbiden = set(biden_users)

In [ ]:
sanders_users = []
for comment in sanders_sub.comments(limit=1000):
    sanders_users.append(comment.author)
print(len(sanders_users))
print(len(set(sanders_users)))

In [ ]:
setsanders = set(sanders_users)

In [ ]:
pete_users = []
for comment in pete_sub.comments(limit=1000):
    pete_users.append(comment.author)
print(len(pete_users))
print(len(set(pete_users)))

In [ ]:
setpet = set(pete_users)

In [ ]:
setnotyang = set(pete_users+sanders_users+biden_users+harris_users)

In [ ]:
len(setnotyang)

In [ ]:
count = 0
for user in setyang:
    if user in setnotyang:
        print(user)
        if user in setbiden:
            print('+biden')
        if user in setharris:
            print('+harris')
        if user in setsanders:
            print('+sanders')
        if user in setpet:
            print('+pete')
        count += 1

In [ ]:
print(count)

Okay, feeling good, out of the 505 most recent yang posters, only 4 posted to one of these other subs, checking other subs

In [ ]:

for user in setbiden:
    if user in setharris:
        print('+harris')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')


In [ ]:

for user in setharris:
    if user in setbiden:
        print('+biden')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')

In [ ]:
count = 0
for user in setpet:
    if user in setsanders:
        count += 1
        
print(count)

In [ ]:
len(setpet)

In [ ]:
len(setsanders)

In [ ]:
len(set(list(setpet)+list(setsanders)))

In [ ]:
171+277

In [ ]:
#backbone scraping code, idea would be to run this for every user in each subreddits set of users
yang_dict = {}
for user in yang_users[22:25]:
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if dict_sub.get(comment.subreddit.display_name,0) == 0:
            dict_sub[comment.subreddit.display_name] = 1
        else:
            dict_sub[comment.subreddit.display_name] += 1
    yang_dict[user] = dict_sub
#     print(f'finished {user}')
    time.sleep(5)

In [ ]:
yang_dict